# FastAPI

## Libs Install

In [1]:
!pip install uvicorn
!pip install fastapi

In [2]:
!pip install wait4it

## ASGI Notebook Layer

### Aka main.py

In [3]:
import uvicorn
from fastapi import FastAPI

app = FastAPI()#root_path='/api')

def run():
    uvicorn.run(app, port=9992, host='0.0.0.0', root_path='')

### Helper Func for Notebook

In [21]:
# Code in this cell is just for (re)starting the API on a Process, and other compatibility stuff with Jupyter cells.
# Just ignore it!
import uvicorn
from multiprocessing import Process

servers = {}
_api_process = None

def start_api(app=None, port=9992, runNew=True):
    """Stop the API if running; Start the API; Wait until API (port) is available (reachable)"""
    assert port in [9991, 9992, 9993, 9994], f'port has unexpected value {port}'
    def run():
        uvicorn.run(app, port=port, host='0.0.0.0', root_path='')    
        
    _api_process = servers.get(port, None)
    if _api_process:
        _api_process.terminate()
        _api_process.join()
        del servers[port]
    
    if runNew:
        assert (not app is None), 'app is None'
        _api_process = Process(target=run, daemon=True)
        _api_process.start()
        servers[port] = _api_process

## Hello World

In [22]:
import requests

def clientTester():
    r = requests.get("http://localhost:9992/api")
    print("Status code:", r.status_code, r.json())

Následující kód je "klasický" hello world, po spuštění otevřete 

http://localhost:31102/docs

Přestože je výše v kódu definovaný port 9992, je nutné si uvědomit, že ve stacku learning je tento port mapován na 31102.
Pokud máte jiný stack, či jiné prostředí, přízpůsobte si url adresu.

Neopomeňte server ukončit `start_api(runNew=False)`

In [23]:
from fastapi import FastAPI

app = FastAPI()#root_path='/api')

@app.get("/api")
def get_root():
    return {"Hello": "World"}

start_api(app=app)

INFO:     Started server process [23775]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9992 (Press CTRL+C to quit)


INFO:     172.20.0.1:46386 - "GET /docs HTTP/1.1" 200 OK
INFO:     172.20.0.1:46386 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     172.20.0.1:46386 - "GET /api HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [23775]


In [7]:
clientTester()

Status code: 200 {'Hello': 'World'}


In [24]:
start_api(runNew=False)

## JSON path params

In [9]:
import requests

def clientTester():
    r = requests.get("http://localhost:9992/api/1")
    print("Status code:", r.status_code, r.json())

In [10]:
from fastapi import FastAPI

app = FastAPI()#root_path='/api')

@app.get("/api/{id}")
def get_root(id):
    return {"Hello": "World", "id": id}

start_api(app=app)

INFO:     Started server process [16763]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9992 (Press CTRL+C to quit)


INFO:     127.0.0.1:48678 - "GET /api/1 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [16763]


In [11]:
clientTester()

Status code: 200 {'Hello': 'World', 'id': '1'}


In [12]:
start_api(runNew=False)

## JSON body params

In [13]:
import requests

def clientTester():
    data = {"name": "World", "description": "whole world"}
    headers = {'Content-type': 'application/json'}
    r = requests.post("http://localhost:9992/api", json=data)
    print("Status code:", r.status_code, r.json())

In [14]:
from fastapi import FastAPI
from typing import Union
from pydantic import BaseModel

class Item(BaseModel):
    name: str
    description: str = None
    
app = FastAPI()#root_path='/api')

@app.post("/api")
async def get_root(item: Item):
    return {"item.name": item.name, "item.description": item.description}

start_api(app=app)

INFO:     Started server process [16783]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9992 (Press CTRL+C to quit)


INFO:     127.0.0.1:48704 - "POST /api HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [16783]


In [15]:
clientTester()

Status code: 200 {'item.name': 'World', 'item.description': 'whole world'}


In [16]:
start_api(runNew=False)

## GQL proxy

In [19]:
import aiohttp
from typing import Union
from pydantic import BaseModel
from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse


class Item(BaseModel):
    query: str
    variables: dict = None
    operationName: str = None

app = FastAPI()#root_path='/api')

proxy = "http://localhost:9993/gql"
@app.post("/gql", response_class=JSONResponse)
async def apigql_post(data: Item, request: Request):
    gqlQuery = {"query": data.query}
    if (data.variables) is not None:
        gqlQuery["variables"] = data.variables
    if (data.operationName) is not None:
        gqlQuery["operationName"] = data.operationName

    # print(demoquery)
    headers = request.headers
    print(headers)
    print(headers.__dict__)
    headers = {}
    async with aiohttp.ClientSession() as session:
        async with session.post(proxy, json=gqlQuery, headers=headers) as resp:
            # print(resp.status)
            json = await resp.json()
    return JSONResponse(content=json, status_code=resp.status)

start_api(app=app)

INFO:     Started server process [22338]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9992 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [22338]


In [20]:
start_api(runNew=False)